In [ ]:
# Upload your Google Cloud service account key
from google.colab import files
# Upload the JSON key file
uploaded = files.upload()

Saving elite-avatar-332412-ad8cb791596c.json to elite-avatar-332412-ad8cb791596c.json


In [ ]:
# Import necessary libraries
!pip install datasets transformers scikit-learn
!pip install transformers scikit-learn google-cloud-storage
import os
import re
import requests
import torch
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import storage
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, load_dataset, load_from_disk
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import DatasetDict
from sklearn.metrics import classification_report


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Set the path to your credentials JSON file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'elite-avatar-332412-ad8cb791596c.json'

# Define GCS bucket and file paths
bucket_name = 'chatbot-participedia'
cases_file_path = 'data/raw data/cases.csv'
methods_file_path = 'data/raw data/methods.csv'
organizations_file_path = 'data/raw data/organisations.csv'

# Initialize Google Cloud Storage client
storage_client = storage.Client()

# Function to load dataset from GCS
def load_dataset_from_gcs(bucket_name, file_path):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    data = blob.download_as_text()  # Download data as text (CSV/Excel/etc.)
    return pd.read_csv(StringIO(data))  # Convert to DataFrame

# Load the datasets
cases = load_dataset_from_gcs(bucket_name, cases_file_path)
methods = load_dataset_from_gcs(bucket_name, methods_file_path)
organizations = load_dataset_from_gcs(bucket_name, organizations_file_path)

In [ ]:

import requests
from bs4 import BeautifulSoup

# Function to scrape the content from the URL and structure the data as a dictionary of headings and their following paragraphs
def scrape_data_from_url(url):
    try:
        # Request the page content
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract headings (h1, h2, h3, etc.)
        headings = []
        for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
            headings.append(tag.get_text(strip=True))

        # Extract paragraphs (p tags)
        paragraphs = []
        for p_tag in soup.find_all('p'):
            paragraphs.append(p_tag.get_text(strip=True))

        # Create a dictionary to store headings and corresponding paragraphs
        content_dict = {}

        # Initialize variables for tracking current heading and paragraphs
        current_heading = None
        current_paragraphs = []

        # Iterate through all the elements (headings and paragraphs)
        all_elements = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p'])

        for element in all_elements:
            # If the element is a heading, assign the previous paragraphs to the current heading
            if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                if current_heading:  # If there was a previous heading, store its paragraphs
                    content_dict[current_heading] = current_paragraphs
                # Update the current heading and reset paragraphs
                current_heading = element.get_text(strip=True)
                current_paragraphs = []
            # If the element is a paragraph, add it to the list of paragraphs for the current heading
            elif element.name == 'p' and current_heading:
                current_paragraphs.append(element.get_text(strip=True))

        # Don't forget to add the last collected heading and paragraphs to the dictionary
        if current_heading:
            content_dict[current_heading] = current_paragraphs

        return content_dict

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Apply the scraping function to DataFrame columns with URLs
cases['url_data'] = cases['url'].apply(scrape_data_from_url)
methods['url_data'] = methods['url'].apply(scrape_data_from_url)
organizations['url_data'] = organizations['url'].apply(scrape_data_from_url)


KeyboardInterrupt: 

In [ ]:
# Save cleaned data back to GCS
def upload_dataframe_to_gcs(dataframe, bucket_name, gcs_path):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    csv_data = dataframe.to_csv(index=False)
    blob.upload_from_string(csv_data, 'text/csv')
    print(f"Uploaded {gcs_path} to GCS successfully.")

data_folder = 'data/scarped url data/'
upload_dataframe_to_gcs(cases, bucket_name, os.path.join(data_folder, 'cases.csv'))
upload_dataframe_to_gcs(methods, bucket_name, os.path.join(data_folder, 'methods.csv'))
upload_dataframe_to_gcs(organizations, bucket_name, os.path.join(data_folder, 'organizations.csv'))

Uploaded data/scarped url data/cases.csv to GCS successfully.
Uploaded data/scarped url data/methods.csv to GCS successfully.
Uploaded data/scarped url data/organizations.csv to GCS successfully.


In [ ]:
# Define GCS bucket and file paths
bucket_name = 'chatbot-participedia'
cases_file_path = 'data/scarped url data/cases.csv'
methods_file_path = 'data/scarped url data/methods.csv'
organizations_file_path = 'data/scarped url data/organizations.csv' # Changed organisations to organizations

# Initialize Google Cloud Storage client
storage_client = storage.Client()

# Function to load dataset from GCS
def load_dataset_from_gcs(bucket_name, file_path):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    data = blob.download_as_text()  # Download data as text (CSV/Excel/etc.)
    return pd.read_csv(StringIO(data))  # Convert to DataFrame

# Load the datasets
cases = load_dataset_from_gcs(bucket_name, cases_file_path)
methods = load_dataset_from_gcs(bucket_name, methods_file_path)
organizations = load_dataset_from_gcs(bucket_name, organizations_file_path)

In [ ]:
!pip install sentence-transformers
!pip install scikit-learn

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pickle
import ast

# Load the pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to create chunks from the dictionary (heading: data)
def create_chunks_from_dict(data_dict, category, url):
    chunks = []
    for heading, paragraphs in data_dict.items():
        # Combine heading with its corresponding paragraphs into one chunk
        chunk = {
            'category': category,
            'url': url,
            'heading': heading,
            'data': ' '.join(paragraphs) if isinstance(paragraphs, list) else str(paragraphs)
        }
        chunks.append(chunk)
    return chunks

# Function to get embeddings for a list of texts
def get_embeddings(texts):
    return model.encode(texts, convert_to_tensor=True)

# Function to safely convert stringified dictionaries back to dictionaries
def convert_to_dict(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)  # Convert string to dictionary
        except (ValueError, SyntaxError):
            print(f"Invalid string format: {value}")
            return None  # Handle invalid strings gracefully
    return value  # Return value as-is if it's already a dictionary

# Function to process data and create chunks
def process_data_and_get_chunks(df, category):
    all_chunks = []
    for idx, row in df.iterrows():
        # Skip rows with NaN in 'url_data'
        if pd.isna(row['url_data']):
            continue

        # Convert stringified dictionaries if needed
        url_data = convert_to_dict(row['url_data'])
        if isinstance(url_data, dict):
            url = row['url']  # Assuming 'url' is another column in the dataframe
            chunks = create_chunks_from_dict(url_data, category, url)
            all_chunks.extend(chunks)
    return all_chunks

# Example: process data from 'cases', 'methods', and 'organizations'
cases_chunks = process_data_and_get_chunks(cases, 'Cases')
methods_chunks = process_data_and_get_chunks(methods, 'Methods')
organizations_chunks = process_data_and_get_chunks(organizations, 'Organizations')

# Combine all chunks
all_chunks = cases_chunks + methods_chunks + organizations_chunks

# Ensure there are valid chunks to process
if all_chunks:
    # Create embeddings for all chunks
    chunk_data = [chunk['data'] for chunk in all_chunks]
    chunk_embeddings = get_embeddings(chunk_data)

    # Save the embeddings and chunk data for later use
    with open('chunk_embeddings.pkl', 'wb') as f:
        pickle.dump({
            'embeddings': chunk_embeddings,
            'chunks': all_chunks
        }, f)

    print(f"Embeddings and chunk data have been saved successfully.")
else:
    print("No valid chunks found. Please check your data.")

import os
from google.cloud import storage

# Function to upload a file to GCS
def upload_file_to_gcs(file_path, bucket_name, gcs_path):
    # Initialize the storage client
    storage_client = storage.Client()

    # Specify the bucket and blob
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)

    # Upload the file
    blob.upload_from_filename(file_path)
    print(f"Uploaded {file_path} to GCS at {gcs_path} successfully.")

# Path to the saved embeddings and chunks file
pkl_file_path = 'chunk_embeddings.pkl'

# Specify your GCS bucket name and desired folder
bucket_name = 'chatbot-participedia'
gcs_folder = 'data/scraped_embeddings/'

# Construct the GCS path
gcs_pkl_path = os.path.join(gcs_folder, 'chunk_embeddings.pkl')

# Upload the .pkl file to GCS
upload_file_to_gcs(pkl_file_path, bucket_name, gcs_pkl_path)
print("uploaded")


Embeddings and chunk data have been saved successfully.
Uploaded chunk_embeddings.pkl to GCS at data/scraped_embeddings/chunk_embeddings.pkl successfully.
uploaded


In [ ]:
import pickle
from google.cloud import storage
import os

# Function to download a file from GCS
def download_file_from_gcs(bucket_name, gcs_path, local_path):
    # Initialize the storage client
    storage_client = storage.Client()

    # Specify the bucket and blob
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)

    # Download the file to local path
    blob.download_to_filename(local_path)
    print(f"Downloaded {gcs_path} from GCS to {local_path} successfully.")

# Define the local path where the embeddings file will be stored
local_pkl_path = 'chunk_embeddings.pkl'

# Specify the GCS bucket name and the file path in GCS
bucket_name = 'chatbot-participedia'
gcs_pkl_path = 'data/scraped_embeddings/chunk_embeddings.pkl'

# Download the .pkl file from GCS
download_file_from_gcs(bucket_name, gcs_pkl_path, local_pkl_path)

# Load the embeddings and chunks from the downloaded .pkl file
with open(local_pkl_path, 'rb') as f:
    data = pickle.load(f)

# Access the embeddings and chunks
embeddings = data['embeddings']
chunks = data['chunks']

# Print the number of chunks and shape of embeddings
print(f"Number of chunks: {len(chunks)}")
print(f"Shape of embeddings: {embeddings.shape if hasattr(embeddings, 'shape') else 'Not a tensor'}")


Downloaded data/scraped_embeddings/chunk_embeddings.pkl from GCS to chunk_embeddings.pkl successfully.
Number of chunks: 63499
Shape of embeddings: torch.Size([63499, 384])


In [ ]:
!pip install fuzzywuzzy
!pip install matplotlib


In [ ]:
import pandas as pd
import pickle
from io import StringIO
from google.cloud import storage
from fuzzywuzzy import fuzz
import time

# Define GCS bucket and file paths
bucket_name = 'chatbot-participedia'
cases_file_path = 'data/scarped url data/cases.csv'
methods_file_path = 'data/scarped url data/methods.csv'
organizations_file_path = 'data/scarped url data/organizations.csv'

# Initialize Google Cloud Storage client
storage_client = storage.Client()

# Function to load dataset from GCS
def load_dataset_from_gcs(bucket_name, file_path):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    data = blob.download_as_text()  # Download data as text (CSV format)
    return pd.read_csv(StringIO(data))  # Convert to DataFrame

# Load the datasets
def load_dataframes():
    try:
        cases_df = load_dataset_from_gcs(bucket_name, cases_file_path)
        methods_df = load_dataset_from_gcs(bucket_name, methods_file_path)
        orgs_df = load_dataset_from_gcs(bucket_name, organizations_file_path)
    except Exception as e:
        print(f"Error loading dataframes: {e}")
        return None, None, None
    return cases_df, methods_df, orgs_df

# Function to search general_issues columns in a dataframe
def search_general_issues(query, dataframe, threshold=80):
    if dataframe is None:
        return pd.DataFrame()
    issue_columns = [col for col in dataframe.columns if 'general_issue' in col.lower()]
    if not issue_columns:
        return pd.DataFrame()

    def fuzzy_match(row):
        for col in issue_columns:
            if fuzz.ratio(query.lower(), str(row[col]).lower()) >= threshold:
                return True
        return False

    filtered_df = dataframe[dataframe.apply(fuzzy_match, axis=1)]
    return filtered_df


# Function to load saved chunk data
def load_saved_data():
    with open('chunk_embeddings.pkl', 'rb') as f:
        saved_data = pickle.load(f)
        all_chunks = saved_data['chunks']
    return all_chunks

# Function to find matches using fuzzy logic
def find_matches(query, data, threshold=80):
    paragraphs = data.split('. ')
    matching_paragraphs = []
    for para in paragraphs:
        if query.lower() in para.lower() or fuzz.ratio(query.lower(), para.lower()) >= threshold:
            matching_paragraphs.append(para)
    return matching_paragraphs

# Function to search across saved chunk data
def search_saved_chunks(query, top_n=10):
    all_chunks = load_saved_data()
    matching_chunks = []
    for chunk in all_chunks:
        matching_paragraphs = find_matches(query, chunk['data'])
        if matching_paragraphs:
            matching_chunks.append((chunk, matching_paragraphs))
    matching_chunks.sort(key=lambda x: len(x[1]), reverse=True)
    return matching_chunks[:top_n]

# Extended chatbot response function
def chatbot_response(user_query, top_n=10):
    # Check for cases in a specific province before other queries
    if "cases in province" in user_query.lower():
        province = user_query.lower().split("cases in province")[-1].strip()
        cases_df, methods_df, orgs_df = load_dataframes()
        if cases_df is not None and 'province' in cases_df.columns:
            # Filter cases based on the province
            province_cases = cases_df[cases_df['province'].str.contains(province, case=False, na=False)]
            if not province_cases.empty:
                # Build a response with case title and URL
                case_info = "\n".join([f"{row['title']}\nLink: {row['url']}" for _, row in province_cases.iterrows()])
                return f"Here are the cases in {province}:\n{case_info}"
            return f"No cases found in {province}."
        else:
            return "Province data not available in the cases dataset."

    # Check for greetings
    current_hour = time.localtime().tm_hour
    greeting = "Good morning!" if 5 <= current_hour < 12 else (
        "Good afternoon!" if 12 <= current_hour < 17 else "Good evening!"
    )
    if any(word in user_query.lower() for word in ['hello', 'hi', 'hey']):
        return f"{greeting} How can I assist you today?"

    # Check for help query
    if 'help' in user_query.lower():
        return "I can assist you with finding information on cases, methods, or organizations, as well as other queries. Just ask!"

    # Check for empty query
    if not user_query.strip():
        return "I'm sorry, I didn't catch that. Could you please rephrase your question?"

    # Load dataframes
    cases_df, methods_df, orgs_df = load_dataframes()

    # Handle general issues
    if "general issue" in user_query.lower():
        query_issue = user_query.lower().split("general issue")[-1].strip()
        if "cases" in user_query.lower():
            dataframe, dataset_name = cases_df, "Cases"
        elif "methods" in user_query.lower():
            dataframe, dataset_name = methods_df, "Methods"
        elif "organizations" in user_query.lower():
            dataframe, dataset_name = orgs_df, "Organizations"
        else:
            return "Please specify if you are looking for cases, methods, or organizations."

        matching_df = search_general_issues(query_issue, dataframe)
        if not matching_df.empty:
            case_info = "\n".join([f"{row['title']}\nLink: {row['url']}" for _, row in matching_df.iterrows()])
            return f"Here are the results for general issue '{query_issue}' in {dataset_name}:\n{case_info}"
        return "No matches found for the specified issue."

    # Handle other queries using chunk data
    top_matches = search_saved_chunks(user_query, top_n)
    if top_matches:
        response = f"Here are the top {top_n} matches based on your query:\n"
        for i, (match, matching_paragraphs) in enumerate(top_matches):
            response += f"\nMatch {i + 1}:\n{match['category']}\nLink: {match['url']}\nData:\n"
            response += "\n".join(matching_paragraphs[:3])  # Display only the top 3 matching paragraphs
        return response
    return "I'm sorry, I couldn't find any matches for your query."

# Function to handle user interaction
def chat():
    print("Welcome to the interactive chatbot! Type 'exit' to end the conversation.")
    while True:
        user_query = input("You: ").strip()
        if user_query.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break
        response = chatbot_response(user_query)
        print(f"Chatbot: {response}")

# Run the interactive chatbot
chat()


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Welcome to the interactive chatbot! Type 'exit' to end the conversation.
Chatbot: Here are the cases in alberta:
Edmonton Citizen Panel 2008
Link: https://participedia.net/case/54
Edmonton Citizens' Jury on Internet Voting
Link: https://participedia.net/case/5037
WinterCity Strategy
Link: https://participedia.net/case/5572
Lethbridge Citizens’ Assembly on Councillor Employment and Compensation
Link: https://participedia.net/case/5990
The Alberta Narratives Project
Link: https://participedia.net/case/6223
Citizen Budget Engagement 2020, City of Wetaskiwin 
Link: https://participedia.net/case/6225
Calgary Arts Development Citizens' Reference Panel
Link: https://participedia.net/case/6674
